In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import nest_asyncio
nest_asyncio.apply()


In [2]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

In [4]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)


In [5]:
from llama_index.core.evaluation import FaithfulnessEvaluator


# define evaluator
evaluator = FaithfulnessEvaluator(llm=ollama_llm)



In [ ]:
# query index
query_engine = vector_index.as_query_engine()
response = query_engine.query(
    "List the companies the author worked"
)
print(response)

In [ ]:
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

In [ ]:
eval_result